In [3]:
import mlflow
import os
#rendre silencieux les messages du GIT
os.environ["GIT_PYTHON_REFRESH"] = "quiet"

In [20]:
#définir une expérimentation
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("Fouilles de Données Massives")

<Experiment: artifact_location='mlflow-artifacts:/541795256308840198', creation_time=1673438592181, experiment_id='541795256308840198', last_update_time=1673438592181, lifecycle_stage='active', name='Fouilles de Données Massives', tags={}>

In [21]:
#instanciation d'un run
my_run = mlflow.start_run(run_name = 'test')


In [11]:
#rajouter le champs "metrics" vide pour l'instant
my_run.metrics = {}

#rajout des éléments de performance
my_run.metrics['Accuracy'] = 1
my_run.metrics['Recall'] = 2
my_run.metrics['Precision'] = 3
my_run.metrics['F_Score']= 4



In [16]:
import pandas as pd
import os
import random
os.chdir(r"C:\Users\pauli\Documents\M2\fouille de données\projet\fichiers")

df = pd.read_csv('df_clean.csv', sep=",")
index_train = pd.read_csv('index_train.csv',sep=",")
index_train = index_train["0"].values.tolist()
index_test = pd.read_csv('index_test.csv',sep=",")
index_test = index_test["0"].values.tolist()
#prendre moins de lignes ??? ou l'under sampling suffira ??? 
index_train30 = random.sample(index_train, round(len(index_train)*0.03))
index_test30 = random.sample(index_test, round(len(index_test)*0.03))
y = df.loc[:,"FlagImpaye"]
# Drop the 'FlagImpaye' column
X = df.drop('FlagImpaye', axis=1)
#ytrain
ytrain = y.loc[index_train30,]
#ytest
ytest = y.loc[index_test30,]
#Xtrain
Xtrain = X.loc[index_train30,]
#Xtest
Xtest = X.loc[index_test30,]


from sklearn.tree import DecisionTreeClassifier
param = {"criterion":"gini","max_depth":None,"min_samples_split":2,"min_samples_leaf":1,"max_features":"sqrt"}
dtc = DecisionTreeClassifier()
dtc = dtc.fit(Xtrain, ytrain)


In [17]:
dtc.get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': None,
 'splitter': 'best'}

In [18]:
#rajouter le modèle dans le log
mlflow.sklearn.log_model(dtc,"Modele Arbre")

#artifact
mlflow.log_artifact("guillaume.txt")

#stocker les métriques
mlflow.log_metrics(my_run.metrics)

#paramètres de l'algo
mlflow.log_params(dtc.get_params())

C:\Users\pauli\anaconda3\envs\metzler\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [22]:
#finaliser l'expérimentation
mlflow.end_run()

In [ ]:
#lancer les runs si on veut faire une seule boucle
for modele in liste:
    #le run
    therun = mlflow.start_run(run_name = modele[0])    
    #entraînement du modèle
    themodele = modele[1].fit(XTrain,yTrain)
    #prédiction
    predmodele = themodele.predict(XTest)
    #récupérer les résultats
    therun.metrics = {}
    therun.metrics['Accuracy'] = metrics.accuracy_score(yTest,predmodele)
    therun.metrics['Rappel'] = metrics.recall_score(yTest,predmodele,pos_label='good')
    therun.metrics['Precision'] = metrics.precision_score(yTest,predmodele,pos_label='good')
    #rajouter le modèle dans le log
    mlflow.sklearn.log_model(themodele,modele[0])
    #info sur les donnees
    mlflow.log_artifact("working_conditions.xlsx")
    #stocker les métriques
    mlflow.log_metrics(therun.metrics)
    #paramètres de l'algo
    mlflow.log_params(themodele.get_params())
    #finaliser
    mlflow.end_run()